In [1]:
#Importing dependencies
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

In [2]:
# Find all CSV files in the working directory
csv_files = glob.glob("*.csv")

# Initialize an empty list to store DataFrames
dfs = []

# Loop through all CSV files and read them into DataFrames
for csv_file in csv_files:
    df = pd.read_csv(csv_file, low_memory=False)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv("merged.csv", index=False)

print("CSV files have been successfully merged into 'merged.csv'")

# creating a dataframe from the merged data
df = pd.read_csv('C:\\Users\money\Desktop\Springboard\Capstone 2\Capstone-2\merged.csv', low_memory=False)

CSV files have been successfully merged into 'merged.csv'


In [3]:
#Call the info method on df to see a summary of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280901 entries, 0 to 280900
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             280901 non-null  int64  
 1   country                280833 non-null  object 
 2   description            280901 non-null  object 
 3   designation            197701 non-null  object 
 4   points                 280901 non-null  int64  
 5   price                  258210 non-null  float64
 6   province               280833 non-null  object 
 7   region_1               234594 non-null  object 
 8   region_2               111464 non-null  object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                280900 non-null  object 
 13  winery                 280901 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [4]:
#Call the head method on df to print the first several rows of the data
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [5]:
#Count (using `.sum()`) the number of missing values (`.isnull()`) in each column of 
#df as well as the percentages (using `.mean()` instead of `.sum()`).
#Order them (increasing or decreasing) using sort_values
#Call `pd.concat` to present these in a single table (DataFrame) with the helpful column names 'count' and '%'
missing = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
Unnamed: 0,0,0.000000
description,0,0.000000
points,0,0.000000
winery,0,0.000000
variety,1,0.000356
country,68,0.024208
province,68,0.024208
price,22691,8.077935
region_1,46307,16.485167
designation,83200,29.618976


There are a number of columns here that are missing a significant amount (> 50%) of data, specifically the title (which is the name and vintage of the wine), region_2 (a secondary designetion of the region where the wine was produced), taster_name (the persone who did the review) and taster_twitter_handle. The designation is also missing nearly 30% of its values, and region_1 is missing more than 16%. The price column, which is critical to answering the business question, is missing roughly 8% of its data. A determination will need to be made, in line with the business problem, if these columns can or should be dropped.

In [6]:
#Let's see if all the entries are unique or if there might be duplicates. We'll use the description 
#label to make that determiniation.
df['description'].value_counts().head()

A little bit funky and unsettled when you pop the screwcap, but soon it finds its floral, blueberry base. Remains superficial and sweet in the mouth, with candied flavors, vanilla and mild oak. Highly regular; could use more concentration and density.    7
A dusty, cigar-smoke character on the nose of this blend of 52% Syrah and 48% Cabernet Sauvignon spices up aromas of strawberry fruit and rotting roses. The palate combines complex cranberry fruits with Kalamata olive, thyme and pencil lead.              6
This is very light, in both flavor and body. The aromas are earthy, with a faint suggestion of blackberries, and the mouthfeel is thin. These aren't terrible flaws, but the wine just lacks substance.                                                        6
92-94 Barrel sample. A rounded wine, its tannins submerged into the ripe fruits. It feels soft, and there is just a bite of alcohol. The structure is soft, generous, opulent.                                                       

It appears that there are many duplicate rows to contend with.

In [8]:
#Are there still duplicates if we take the price column into account?
df1 =  df.astype({'price':'str'})
(df1['price'] + ', ' + df['description']).value_counts().head()

40.0, A dusty, cigar-smoke character on the nose of this blend of 52% Syrah and 48% Cabernet Sauvignon spices up aromas of strawberry fruit and rotting roses. The palate combines complex cranberry fruits with Kalamata olive, thyme and pencil lead.                                                                                                                       6
nan, A fine and impressively complex wine, rich, packed with sultanas and dark chocolate. Very firm, this is worth aging but drinkable now.                                                                                                                                                                                                                                   6
70.0, Possibly just in a sullen phase, this reserve bottling from Craggy's home vineyard is dominated for the moment by smoke and cedar on the nose, and while there is some cherry fruit apparent in the mouth, it doesn't appear to be fully expressing itself. The im

Yes, still many duplicates.

In [9]:
#Because the varietal determines if we are talking about red, white, or dessert wines we need to see 
#the breakdown of varieties.
df['variety'].value_counts()

Pinot Noir                  27563
Chardonnay                  26235
Cabernet Sauvignon          22272
Red Blend                   19008
Bordeaux-style Red Blend    14262
                            ...  
Kotsifali                       1
Petit Courbu                    1
Misket                          1
Babosa Negro                    1
Carnelian                       1
Name: variety, Length: 756, dtype: int64

The data looks to be overwhelmingly red varietals.

In [10]:
#How many different varietals are we actually dealing with?
df['variety'].nunique()

756

In [11]:
#What about the price data? Let's look at the statistics around that column.
price_stats = df['price'].describe()
print(price_stats)

count    258210.000000
mean         34.177162
std          38.611708
min           4.000000
25%          16.000000
50%          25.000000
75%          40.000000
max        3300.000000
Name: price, dtype: float64


It apprears there is a large variation in price, from a minimum of $4 to a maximum of $3300. It might also be worth noting that the mean is closer to the 75